In [ ]:
""""
After annotating the clothing images using VGG16/ Resnet50/ InceptionV3 based- models, the received data are the input for learning the compatibility.

This is the preparation for compatibility learning using BPR model, applying Polyvore's annotation and self-mapped category of clothing item attributes and which part does that attribute take part on body (top/ bottom).
To be more specific, the work below are:
- Transform the labeled data 
-> Build the BPR model to learn the pair score (between top-bottom clothing item attributes) 
-> Make the recommendation based on the pair scores

""""

from google.colab import drive
import os
import pandas as pd

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Practicum- ChauDao')

Mounted at /content/drive


In [ ]:
df_train = pd.read_json (r'./Polyvore/train_no_dup_new_100.json')
df_test = pd.read_json (r'./Polyvore/test_no_dup_new_100.json')

In [ ]:
category_list_tb = pd.read_csv (r'./Polyvore/Filtered_top_bottom/category_summarize_top_bottom.csv')
#category_summarize_top_bottom.csv: manually mapping the original category of Polyvore dataset to top/bottom

##1. Remove cats not top/bottom
output: df_train_tb_fin, df_test_tb_fin

In [ ]:
from tqdm import tqdm

In [ ]:
#append df.items_category: remove item that not declare in category_list_tb
#df_train
df_train_tb = pd.DataFrame(columns=['set_id', 'items_category','new_items_category'])
cat = list(category_list_tb["id"])
data = []

for set_id in tqdm(df_train.set_id.unique()):
  items_category = df_train[(df_train.set_id == set_id)].items_category.values
  
  old = items_category[0][:]
  new_items_category = []
  
  for i in range(len(old)):
    if old[i] in cat:
      new_items_category.append(old[i])

  data.append({'set_id': set_id, 'items_category': items_category[0][:], 'new_items_category': new_items_category})

df_train_tb = df_train_tb.append(data, ignore_index=True)

In [ ]:
#append df.items_category: remove item that not declare in category_list_tb
#df_test
df_test_tb = pd.DataFrame(columns=['set_id', 'items_category','new_items_category'])
cat = list(category_list_tb["id"])
data = []

for set_id in tqdm(df_test.set_id.unique()):
  items_category = df_test[(df_test.set_id == set_id)].items_category.values
  
  old = items_category[0][:]
  new_items_category = []
  
  for i in range(len(old)):
    if old[i] in cat:
      new_items_category.append(old[i])

  data.append({'set_id': set_id, 'items_category': items_category[0][:], 'new_items_category': new_items_category})

df_test_tb = df_test_tb.append(data, ignore_index=True)

In [ ]:
df_test_tb

In [ ]:
df_train_tb_fin = pd.DataFrame(columns=['set_id', 'items_category','new_items_category'])

In [ ]:
##Remove set_id having number of items <=1
#train
data = []
for set_id in tqdm(df_train_tb.set_id.unique()):
  items_category = df_train_tb[(df_train_tb.set_id == set_id)].items_category.values
  new_items_category = df_train_tb[(df_train_tb.set_id == set_id)].new_items_category.values
  new_ic = new_items_category[0][:]
  if len(new_ic) >1:
    data.append({'set_id': set_id, 'items_category': items_category[0][:], 'new_items_category': new_items_category[0][:]})

df_train_tb_fin = df_train_tb_fin.append(data, ignore_index=True)

In [ ]:
##Remove set_id having number of items <=1
#test
df_test_tb_fin = pd.DataFrame(columns=['set_id', 'items_category','new_items_category'])

data = []
for set_id in tqdm(df_test_tb.set_id.unique()):
  items_category = df_test_tb[(df_test_tb.set_id == set_id)].items_category.values
  new_items_category = df_test_tb[(df_test_tb.set_id == set_id)].new_items_category.values
  new_ic = new_items_category[0][:]
  if len(new_ic) >1:
    data.append({'set_id': set_id, 'items_category': items_category[0][:], 'new_items_category': new_items_category[0][:]})

df_test_tb_fin = df_test_tb_fin.append(data, ignore_index=True)

In [ ]:
df_train_tb_fin.head(5)

In [ ]:
import pickle

In [ ]:
df_train_tb_fin.to_pickle("./Polyvore/Filtered_top_bottom/df_train_tb_fin.pkl")
df_test_tb_fin.to_pickle("./Polyvore/Filtered_top_bottom/df_test_tb_fin.pkl")

##2. Edit df => structure: top, bottom in pair

###view number of set include item in group 3 (top1_bottom2_full3)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
import plotly
import plotly.graph_objs as go



In [ ]:
df1 = pd.DataFrame() 
result = [] 

for set_id in tqdm(df_train_tb_fin.set_id.unique()):
  list1 = df_train_tb_fin.loc[df_train_tb_fin['set_id'] == set_id, 'new_items_category'].values[0]
  str1 = ' '.join(str(e) for e in list1)
  result.append({"set_id": set_id,"str": str1})
df1 = df1.append(result, ignore_index=True)

In [ ]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(0, 1)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
common_words = get_top_n_bigram(df1['str'], 41)
for word, freq in common_words:
    print(word, freq)
df3 = pd.DataFrame(common_words, columns = ['pair' , 'count'])
df3.groupby('pair').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', ylabel='Count', title='Top 20 unigrams')
## dont have set id with item_category in (3,4,5): group 3

###split new_items_category into 2 new columns: top, bottom

In [ ]:
df_test_tb_fin.head(5)

In [ ]:
top = category_list_tb.loc[category_list_tb['top1_bottom2_full3'] == 1, ('id','name','top1_bottom2_full3')]

In [ ]:
bottom = category_list_tb.loc[category_list_tb['top1_bottom2_full3'] == 2, ('id','name','top1_bottom2_full3')]

In [ ]:
df_train_tb_fin

In [ ]:
df_train = pd.DataFrame(columns=['set_id', 'new_items_category','top','bottom'])
cat_top = list(top["id"])
cat_bottom = list(bottom["id"])
data = []

for set_id in tqdm(df_train_tb_fin.set_id.unique()):
  items_category = df_train_tb_fin[(df_train_tb_fin.set_id == set_id)].new_items_category.values
  
  old = items_category[0][:]
  top_list = []
  bottom_list = []
  
  for i in range(len(old)):
    if old[i] in cat_top:
      top_list.append(old[i])
    if old[i] in cat_bottom:
      bottom_list.append(old[i])

  if len(top_list)>=1 and len(bottom_list)>=1:
    data.append({'set_id': set_id, 'new_items_category': items_category[0][:], 'top': top_list,'bottom': bottom_list})

df_train = df_train.append(data, ignore_index=True)

In [ ]:
df_train#9948 rows

In [ ]:
df_test = pd.DataFrame(columns=['set_id', 'new_items_category','top','bottom'])
cat_top = list(top["id"])
cat_bottom = list(bottom["id"])
data = []

for set_id in tqdm(df_test_tb_fin.set_id.unique()):
  items_category = df_test_tb_fin[(df_test_tb_fin.set_id == set_id)].new_items_category.values
  
  old = items_category[0][:]
  top_list = []
  bottom_list = []
  
  for i in range(len(old)):
    if old[i] in cat_top:
      top_list.append(old[i])
    if old[i] in cat_bottom:
      bottom_list.append(old[i])

  if len(top_list)>=1 and len(bottom_list)>=1:
    data.append({'set_id': set_id, 'new_items_category': items_category[0][:], 'top': top_list,'bottom': bottom_list})

df_test = df_test.append(data, ignore_index=True)

In [ ]:
df_test#1211 rows

In [ ]:
df_train.to_pickle("./Polyvore/Filtered_top_bottom/train/df_train.pkl")
df_test.to_pickle("./Polyvore/Filtered_top_bottom/test/df_test.pkl")

### split each top to each bottom

In [ ]:
from tqdm import tqdm 

In [ ]:
df_train = pd.read_pickle ('./Polyvore/Filtered_top_bottom/train/df_train.pkl')
df_test = pd.read_pickle ('./Polyvore/Filtered_top_bottom/test/df_test.pkl')

In [ ]:
import itertools  

#train 
df_train_split = pd.DataFrame() 

for set_id in tqdm(df_train.set_id.unique()):
  #x = df[df['set_id'] == set_id].index
  A = df_train.loc[df_train['set_id'] == set_id, 'top'].values[0]
  B = df_train.loc[df_train['set_id'] == set_id, 'bottom'].values[0]
  c = list(itertools.product(A, B))
  temp = pd.DataFrame(c,columns =['top', 'bottom'])
  temp['set_id'] = set_id
  df_train_split = df_train_split.append(temp)

    

In [ ]:
df_train_split.head(5)

In [ ]:
#test
df_test_split = pd.DataFrame() 

for set_id in tqdm(df_test.set_id.unique()):
  #x = df[df['set_id'] == set_id].index
  A = df_test.loc[df_test['set_id'] == set_id, 'top'].values[0]
  B = df_test.loc[df_test['set_id'] == set_id, 'bottom'].values[0]
  c = list(itertools.product(A, B))
  temp = pd.DataFrame(c,columns =['top', 'bottom'])
  temp['set_id'] = set_id
  df_test_split = df_test_split.append(temp)

### update to new category list (range(0:n))

In [ ]:
import numpy as np

In [ ]:
sorted(df_train_split.top.unique())

In [ ]:
category_list_tb = pd.read_csv (r'./Polyvore/Filtered_top_bottom/category_summarize_top_bottom.csv')

In [ ]:
category_list_tb.head(5)

In [ ]:
unique_top = df_train_split.top.unique()
top_ids = dict(zip(unique_top, np.arange(unique_top.shape[0], dtype=np.int32)))

unique_bottom = df_train_split.bottom.unique()
bottom_ids = dict(zip(unique_bottom, np.arange(unique_bottom.shape[0], dtype=np.int32)))

In [ ]:
bottom_ids

In [ ]:
bottom = pd.DataFrame(bottom_ids.items(), columns=['old', 'new'])
top = pd.DataFrame(top_ids.items(), columns=['old', 'new'])

In [ ]:
top.to_pickle("./Polyvore/Filtered_top_bottom/top_categories.pkl")
bottom.to_pickle("./Polyvore/Filtered_top_bottom/bottom_categories.pkl")

In [ ]:
top

In [ ]:
df_train_split['top_id'] = df_train_split.top.apply(lambda u: top_ids[u])
df_train_split['bottom_id'] = df_train_split.bottom.apply(lambda m: bottom_ids[m])

In [ ]:
df_train_split

In [ ]:
sorted(df_test_split.top.unique())

In [ ]:
df_test_split['top_id'] = df_test_split.top.apply(lambda u: top_ids[u])
df_test_split['bottom_id'] = df_test_split.bottom.apply(lambda m: bottom_ids[m])

In [ ]:
df_test_split

In [ ]:
df_train_split.to_pickle("./Polyvore/Filtered_top_bottom/train/df_train_split_map.pkl")
df_test_split.to_pickle("./Polyvore/Filtered_top_bottom/test/df_test_split_map.pkl")

## 3. Create file df_triplest

In [ ]:
from tqdm import tqdm
import numpy as np

In [ ]:
df_train_triplest = pd.DataFrame(columns=['set_id', 'positive_item_id', 'negative_item_id'])
# set_id = top_id; positive_item_id = positive_bottom_id, negative_item_id = negative_item_id

cat = list(bottom["new"]) #np.arange(0,14, 1)
data = []

for top_id in tqdm(df_train_split.top_id.unique()):
  positive_items = df_train_split[(df_train_split.top_id == top_id)].bottom_id.unique()

  pos = positive_items[:]
  negative_items = cat
  for i in range(len(pos)):
      element = pos[i]
      c = negative_items
      negative_items = [c[k] for k in range(len(c)) if c[k] != element ]

  data.append({'set_id': top_id, 'positive_item_id': positive_items[:], 'negative_item_id': negative_items})
df_train_triplest = df_train_triplest.append(data, ignore_index=True)



In [ ]:
df_train_triplest

In [ ]:
df_test_triplest = pd.DataFrame(columns=['set_id', 'positive_item_id', 'negative_item_id'])
# set_id = top_id; positive_item_id = positive_bottom_id, negative_item_id = negative_item_id

cat = list(bottom["new"]) #np.arange(0,14, 1)
data = []

for top_id in tqdm(df_test_split.top_id.unique()):
  positive_items = df_test_split[(df_test_split.top_id == top_id)].bottom_id.unique()

  pos = positive_items[:]
  negative_items = cat
  for i in range(len(pos)):
      element = pos[i]
      c = negative_items
      negative_items = [c[k] for k in range(len(c)) if c[k] != element ]

  data.append({'set_id': top_id, 'positive_item_id': positive_items[:], 'negative_item_id': negative_items})
df_test_triplest = df_test_triplest.append(data, ignore_index=True)



In [ ]:
df_test_triplest

In [ ]:
df_train_triplest.to_pickle("./Polyvore/Filtered_top_bottom/train/df_train_triplest.pkl")
df_test_triplest.to_pickle("./Polyvore/Filtered_top_bottom/test/df_test_triplest.pkl")

In [ ]:
import itertools  

df_train_triplest_fin = pd.DataFrame() 

for set_id in tqdm(df_train_triplest.set_id.unique()):
  #x = df[df['set_id'] == set_id].index
  A = df_train_triplest.loc[df_train_triplest['set_id'] == set_id, 'positive_item_id'].values[0]
  B = df_train_triplest.loc[df_train_triplest['set_id'] == set_id, 'negative_item_id'].values[0]
  if len(B)!= 0:
    c = list(itertools.product(A, B))
    temp = pd.DataFrame(c,columns =['positive_item_id', 'negative_item_id'])
    temp['set_id'] = set_id
  df_train_triplest_fin = df_train_triplest_fin.append(temp)

In [ ]:
df_train_triplest_fin.head(5) #4323 rows

In [ ]:
#import itertools  

df_test_triplest_fin = pd.DataFrame() 

for set_id in tqdm(df_test_triplest.set_id.unique()):
  #x = df[df['set_id'] == set_id].index
  A = df_test_triplest.loc[df_test_triplest['set_id'] == set_id, 'positive_item_id'].values[0]
  B = df_test_triplest.loc[df_test_triplest['set_id'] == set_id, 'negative_item_id'].values[0]
  c = list(itertools.product(A, B))
  temp = pd.DataFrame(c,columns =['positive_item_id', 'negative_item_id'])
  temp['set_id'] = set_id
  df_test_triplest_fin = df_test_triplest_fin.append(temp)

In [ ]:
df_test_triplest_fin.head(5)#3776 rows

In [ ]:
df_train_triplest_fin.to_pickle("./Polyvore/Filtered_top_bottom/train/df_train_triplest_fin.pkl")
df_test_triplest_fin.to_pickle("./Polyvore/Filtered_top_bottom/test/df_test_triplest_fin.pkl")

## 4. Build model

In [ ]:
top= pd.read_pickle("./Polyvore/Filtered_top_bottom/top_categories.pkl")
bottom= pd.read_pickle("./Polyvore/Filtered_top_bottom/bottom_categories.pkl")

In [ ]:
df_train_split = pd.read_pickle("./Polyvore/Filtered_top_bottom/train/df_train_split_map.pkl")
df_test_split = pd.read_pickle("./Polyvore/Filtered_top_bottom/test/df_test_split_map.pkl")
df_train_triplest_fin = pd.read_pickle("./Polyvore/Filtered_top_bottom/train/df_train_triplest_fin.pkl")
df_test_triplest_fin = pd.read_pickle("./Polyvore/Filtered_top_bottom/test/df_test_triplest_fin.pkl")

In [ ]:
#frames = [df_train_triplest_fin, df_test_triplest_fin]
  
#df_triplest = pd.concat(frames)
df_triplest = df_train_triplest_fin

In [ ]:
df_triplest.drop_duplicates()
df_triplest

,positive_item_id,negative_item_id,set_id
0,0,12,0
1,1,12,0
2,2,12,0
3,3,12,0
4,8,12,0
...,...,...,...
21,10,14,15
22,5,13,15
23,5,14,15
24,12,13,15


In [ ]:
frames1 = [df_train_split, df_test_split]
  
df_full = pd.concat(frames1)

In [ ]:
df_full.drop_duplicates()
df_full

In [ ]:
unique_users = top.new.unique()
unique_movies = bottom.new.unique() #np.arange(0,34, 1) 33 category

num_users = unique_users.shape[0]
num_items = unique_movies.shape[0]

num_users, num_items

(16, 15)

In [ ]:
unique_movie_ids = list(bottom.new.unique())

In [ ]:
import numpy as np
import pandas as pd
from os import path
from collections import OrderedDict
from tqdm import tqdm
from typing import Dict

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Embedding, Flatten, Input, Lambda
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def bpr_predict(model: Model, user_id: int, item_ids: list, user_layer='user_embedding', item_layer='item_embedding'):
    """
    Predict by multiplication user vector by item matrix
    
    :return: list of the scores
    """
    user_vector = model.get_layer(user_layer).get_weights()[0][user_id]
    item_matrix = model.get_layer(item_layer).get_weights()[0][item_ids]

    scores = (np.dot(user_vector, item_matrix.T))

    return scores

In [ ]:
@tf.function
def identity_loss(_, y_pred):
    return tf.math.reduce_mean(y_pred)

In [ ]:
@tf.function
def bpr_triplet_loss(X: dict):
    """
    Calculate triplet loss - as higher the difference between positive interactions
    and negative interactions as better

    :param X: X contains the user input, positive item input, negative item input
    :return:
    """
    positive_item_latent, negative_item_latent, user_latent = X

    positive_interactions = tf.math.reduce_sum(tf.math.multiply(user_latent, positive_item_latent), axis=-1, keepdims=True)
    negative_interactions = tf.math.reduce_sum(tf.math.multiply(user_latent, negative_item_latent), axis=-1, keepdims=True)

    return tf.math.subtract(tf.constant(1.0), tf.sigmoid(tf.math.subtract(positive_interactions, negative_interactions)))

In [ ]:
def out_shape(shapes):
    return shapes[0]
    

def build_model(num_users: int, num_items: int, latent_dim: int) -> Model:
    """
    Build a model for Bayesian personalized ranking

    :param num_users: a number of the unique users
    :param num_items: a number of the unique movies
    :param latent_dim: vector length for the latent representation
    :return: Model
    """
    user_input = Input((1,), name='user_input')

    positive_item_input = Input((1,), name='positive_item_input')
    negative_item_input = Input((1,), name='negative_item_input')
    # One embedding layer is shared between positive and negative items
    item_embedding_layer = Embedding(num_items, latent_dim, name='item_embedding', input_length=1)

    positive_item_embedding = Flatten()(item_embedding_layer(positive_item_input))
    negative_item_embedding = Flatten()(item_embedding_layer(negative_item_input))

    user_embedding = Embedding(num_users, latent_dim, name='user_embedding', input_length=1)(user_input)
    user_embedding = Flatten()(user_embedding)

    triplet_loss = Lambda(bpr_triplet_loss, output_shape=out_shape)([positive_item_embedding,
                                                             negative_item_embedding,
                                                             user_embedding])

    model = Model(inputs=[positive_item_input, negative_item_input, user_input], outputs=triplet_loss)

    return model

In [ ]:
'''
#1
latent_dim = 32
batch_size = 125 
num_epochs = 1
lr = 0.001

#2
latent_dim = 32
batch_size = 125 
num_epochs = 5 ##
lr = 0.001


#3
latent_dim = 32
batch_size = 125 
num_epochs = 1
lr = 0.01 ##

#4
latent_dim = 32
batch_size = 256 ##
num_epochs = 1
lr = 0.001

#5
latent_dim = 64 ##
batch_size = 256 
num_epochs = 1
lr = 0.001

#6
latent_dim = 32
batch_size = 125 
num_epochs = 10 ##
lr = 0.001

#7
latent_dim = 32
batch_size = 125 
num_epochs = 5
lr = 0.01
'''
#8
latent_dim = 32
batch_size = 125 
num_epochs = 1
lr = 0.1
'''
'''
model = build_model(num_users, num_items, latent_dim)
model.compile(loss=identity_loss, optimizer=Adam(learning_rate=lr))

In [ ]:
trainable_count = np.sum([K.count_params(w) for w in model.trainable_weights])
non_trainable_count = np.sum([K.count_params(w) for w in model.non_trainable_weights])

print('Total number of parameters: {:,}'.format(trainable_count + non_trainable_count))
print('Trainable number of parameters: {:,}'.format(trainable_count))
print('Non-trainable number of parameters: {:,}'.format(non_trainable_count))

print('Training data length: {:,}'.format(df_triplest.shape[0]))

Total number of parameters: 992.0
Trainable number of parameters: 992
Non-trainable number of parameters: 0.0
Training data length: 376


In [ ]:
# fix error Failed to convert a NumPy array to a Tensor (Unsupported object type int)
df_triplest['negative_item_id'] = np.asarray(df_triplest.negative_item_id).astype(np.int64)
df_triplest['positive_item_id'] = np.asarray(df_triplest.positive_item_id).astype(np.int64)

df_triplest.dtypes

positive_item_id    int64
negative_item_id    int64
set_id              int64
dtype: object

In [ ]:
#%%time

X = {
    'user_input': tf.convert_to_tensor(df_triplest.set_id),
    'positive_item_input': tf.convert_to_tensor(df_triplest.positive_item_id),
    'negative_item_input': tf.convert_to_tensor(df_triplest.negative_item_id)
    
}

model.fit(X,tf.ones(df_triplest.shape[0]), batch_size=batch_size,epochs=num_epochs)


4/4 [==============================] - 0s 4ms/step - loss: 0.4467


##Evaluation

### Def function

In [ ]:
def full_auc(model: Model, ground_truth: Dict[int, list], items: list) -> float:
    """
    Measure AUC for model and ground truth for all items
    :param model: 
    :param ground_truth: dictionary of the users and the high ranked movies for the specific user
    :param items: a list of the all available movies
    :return: AUC
    """

    number_of_items = len(items)
    scores = []

    for user_id, true_item_ids in ground_truth:
        predictions = bpr_predict(model, user_id, items)
        grnd = np.zeros(number_of_items, dtype=np.int32)

        for p in true_item_ids:
            index = items.index(p)
            grnd[index] = 1

        if true_item_ids:
          try:
            a = roc_auc_score(grnd, predictions)
          except ValueError:
            pass
          scores.append(a)

    return sum(scores) / len(scores)

In [ ]:
def mean_average_precision_k(model: Model, 
                           ground_truth: Dict[int, list], 
                           items: list, 
                           k=100) -> float:
    """
    Calculate mean eavarage precission per user
    
    :param model: 
    :param ground_truth: dictionary of the users and the high ranked movies for the specific user
    :param items: a list of the all available movies
    :param k: top N recommendations per user
    :return: mean eavarage precission
    """
    scores = []

    for user, actual in ground_truth:
        predictions = bpr_predict(model, user, items)
        predictions = dict(zip(items, predictions))
        predictions = sorted(predictions.items(), key=lambda kv: kv[1], reverse=True)[:k]
        predictions = list(OrderedDict(predictions).keys())

        score = 0.0
        num_hits = 0.0

        for i, p in enumerate(predictions):
            if p in actual:
                num_hits += 1.0
                score += num_hits / (i + 1.0)

        score = score / min(len(actual), k)
        scores.append(score)

    return np.mean(scores)

### prepare ground_truth

In [ ]:
df_train_split = pd.read_pickle("./Polyvore/Filtered_top_bottom/train/df_train_split_map.pkl")
df_test_split = pd.read_pickle("./Polyvore/Filtered_top_bottom/test/df_test_split_map.pkl")

In [ ]:
df_train_split

In [ ]:
from tqdm import tqdm

In [ ]:
# train
ground_truth_train = pd.DataFrame(columns=['top_id', 'bottom_id'])
data = []
for top_id in tqdm(df_train_split.top_id.unique()):
  b = df_train_split[(df_train_split.top_id == top_id)].bottom_id.unique()

  data.append({'top_id': top_id, 'bottom_id': b})
ground_truth_train = ground_truth_train.append(data, ignore_index=True)



In [ ]:
ground_truth_train

In [ ]:
ground_truth_test = pd.DataFrame(columns=['top_id', 'bottom_id'])
data = []
for top_id in tqdm(df_test_split.top_id.unique()):
  b = df_test_split[(df_test_split.top_id == top_id)].bottom_id.unique()

  data.append({'top_id': top_id, 'bottom_id': b})
ground_truth_test = ground_truth_test.append(data, ignore_index=True)

In [ ]:
ground_truth_test

In [ ]:
import pickle

ground_truth_train.to_pickle("./Polyvore/Filtered_top_bottom/train/df_ground_truth_train.pkl")
ground_truth_test.to_pickle("./Polyvore/Filtered_top_bottom/test/ground_truth_test.pkl")

In [ ]:
#Train
##top_id
ground_truth_train['top_id'] = np.asarray(ground_truth_train.top_id).astype(np.int64)
##bottom_id
for i in range(0,16):
  a = ground_truth_train['bottom_id'][i]
  ground_truth_train['bottom_id'][i] = a.tolist()

In [ ]:
#Test
##top_id
ground_truth_test['top_id'] = np.asarray(ground_truth_test.top_id).astype(np.int64)
##bottom_id
for i in range(0,16):
  a = ground_truth_test['bottom_id'][i]
  ground_truth_test['bottom_id'][i] = a.tolist()

In [ ]:
ground_truth_test

In [ ]:
ground_truth_test.dtypes

In [ ]:
import pickle

ground_truth_train.to_pickle("./Polyvore/Filtered_top_bottom/train/df_ground_truth_train.pkl")
ground_truth_test.to_pickle("./Polyvore/Filtered_top_bottom/test/ground_truth_test.pkl")

### Test

In [ ]:
ground_truth_train = pd.read_pickle("./Polyvore/Filtered_top_bottom/train/df_ground_truth_train.pkl")
ground_truth_test = pd.read_pickle("./Polyvore/Filtered_top_bottom/test/ground_truth_test.pkl")

In [ ]:
print(f'AUC train: {full_auc(model, ground_truth_train.values, unique_movie_ids)}')

AUC train: 0.9975961538461539


In [ ]:
print(f'Mean average precision train: {mean_average_precision_k(model, ground_truth_train.values, unique_movie_ids)}')

Mean average precision train: 0.9996565934065934


In [ ]:
print(f'AUC test: {full_auc(model, ground_truth_test.values, unique_movie_ids)}')

AUC test: 0.6804771501646502


In [ ]:
print(f'Mean average precision test: {mean_average_precision_k(model, ground_truth_test.values, unique_movie_ids)}')

Mean average precision test: 0.8646615737678581


## Make RCM:

In [ ]:
top= pd.read_pickle("./Polyvore/Filtered_top_bottom/top_categories.pkl")

### Table with full list of pair score:

In [ ]:
#table with full list of pair score:

df_pair_score =  pd.DataFrame(columns=['top_id', 'bottom_id','pair_score'])
data = []
a = top['new']

for i in range(len(a)) :
  top = a[i]
  pair_score_list =  np.asarray(bpr_predict(model, top, unique_movie_ids)).astype(np.float)
  for j in range(len(pair_score_list)):
    index = j 
    pair_score = pair_score_list[j]
    data.append({'top_id': top,'bottom_id':index, 'pair_score':pair_score})

df_pair_score = df_pair_score.append(data,ignore_index=True)

In [ ]:
df_pair_score

###Table with full bottom_list + its categories

In [ ]:
bottom_img = pd.DataFrame({'img_id': {0: '123_2', 1: '123_5', 2: '123_4', 3: '123_8', 4: '123_7', 5: '123_1', 6: '123_9', 7: '123_3', 8: '123_6'}, 'bottom_category': {0: [0,1,2], 1: [0,2,3], 2: [5,14,1], 3: [9,13,10,2,5,4], 4: [2], 5: [14,1,2,3,4], 6: [6,7], 7: [0,10], 8: [8,9,12]}})

In [ ]:
bottom_img

### Table with input top image's information

In [ ]:
input_top = pd.DataFrame({'img_id': {0: '234_2'},'top_category':{0: [0,2,5,9]}})

In [ ]:
input_top

### Calculate bottom_img_score

In [ ]:
top_category = input_top.top_category[0]
data =[]
output_bottom = pd.DataFrame(columns = ['img_id','img_score'])

for img_id in bottom_img.img_id.unique():
  bottom_category = bottom_img[(bottom_img.img_id == img_id)].bottom_category.values[0]
  score = 0
  img_score = 0
  #top_category = input_top.top_category[0]
  for i in range(0,len(bottom_category)):
    for j in range(0,len(top_category)):
      x = df_pair_score[(df_pair_score.top_id == top_category[j])&(df_pair_score.bottom_id == bottom_category[i])].pair_score.values
      score = score + x
  data.append ({'img_id':img_id,'img_score':score})

output_bottom = output_bottom.append(data,ignore_index=True )




In [ ]:
output_bottom.sort_values(by=['img_score'],ascending = False)

,img_id,img_score
6,123_9,[0.02185550460126251]
0,123_2,[0.015054209623485804]
1,123_5,[0.010595788015052676]
7,123_3,[0.009235450124833733]
2,123_4,[0.007444234448485076]
4,123_7,[0.006190633401274681]
5,123_1,[0.000493430532515049]
8,123_6,[0.00045066187158226967]
3,123_8,[-0.017440810741391033]


###Function

In [ ]:
def full_auc(model: Model, ground_truth: Dict[int, list], items: list) -> float:
    """
    Measure AUC for model and ground truth for all items
    
    :param model: 
    :param ground_truth: dictionary of the users and the high ranked movies for the specific user
    :param items: a list of the all available movies
    :return: AUC
    """

    number_of_items = len(items)
    scores = []

    for user_id, true_item_ids in ground_truth:
        predictions = bpr_predict(model, user_id, items)
        grnd = np.zeros(number_of_items, dtype=np.int32)

        for p in true_item_ids:
            index = items.index(p)
            grnd[index] = 1

        if true_item_ids:
            scores.append(roc_auc_score(grnd, predictions))

    return sum(scores) / len(scores)

In [ ]:
def x_score(bottom_category:list, top_category:list):
  score = 0
  for i in range(0,len(bottom_category)):
      for j in range(0,len(top_category)):
        x = df_pair_score[(df_pair_score.top_id == top_category[j])&(df_pair_score.bottom_id == bottom_category[i])].pair_score.values
        score = score + x
  return score
      

In [ ]:
def img_score(img_id ,bottom_category:list, top_category:list):
  score_list = x_score(bottom_category,top_category)
  img_score = tf.math.reduce_sum(score_list).numpy()
  return img_score

In [ ]:
def make_rcm(input_top):
  top_category = input_top.top_category[0]
  data =[]
  output_bottom = pd.DataFrame(columns = ['img_id','img_score'])


  for img_id in bottom_img.img_id.unique():
    bottom_category = bottom_img[(bottom_img.img_id == img_id)].bottom_category.values[0]
    top_category = input_top.top_category[0]
    total_score = img_score(bottom_category,top_category)
    data.append ({'img_id':img_id,'img_score':total_score})
    output_bottom= output_bottom.append(data,ignore_index=True ).sort_values(by=['img_score'],ascending = False)
    
  return output_bottom.head(5)

In [ ]:
make_rcm(input_top)

TypeError: ignored

In [ ]:
input_top

,img_id,top_category
0,234_2,"[0, 2, 5, 9]"
